In [844]:
import numpy as np
import boto3
import pandas as pd
import random
import io
import json
import math
from pprint import pprint

In [845]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

In [846]:
data = pd.read_csv('all-in-one.csv')

df_reordered_cos = data[['boat_speed','angle_of_attack','wind_speed','cos_target_angle', 'sin_target_angle','cos_boat_angle','sin_boat_angle']]

df_reordered_sin = data[['boat_speed','angle_of_attack','wind_speed','sin_target_angle','cos_target_angle', 'cos_boat_angle','sin_boat_angle']]


# Binary classifier only accepts 0 and 1
#df_reordered['sign_sin_target_angle'] = df_reordered['sign_sin_target_angle'].map({-1:0, 1:1})
#df_reordered['sign_sin_boat_angle'] = df_reordered['sign_sin_boat_angle'].map({-1:0, 1:1})

df_reordered_cos.describe()

,boat_speed,angle_of_attack,wind_speed,cos_target_angle,sin_target_angle,cos_boat_angle,sin_boat_angle
count,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000,2869.000000
mean,5.964360,12.365479,9.157457,-0.184606,0.166315,-0.213446,0.152944
std,2.373046,107.211737,5.706170,0.680841,0.689233,0.653259,0.710370
min,0.000000,-177.061360,2.000000,-1.000000,-0.999998,-1.000000,-0.999970
25%,4.151830,-108.771260,4.995720,-0.911645,-0.452754,-0.834221,-0.616197
50%,5.374240,44.951450,6.975660,-0.291872,0.176061,-0.451541,0.273611
75%,7.753420,95.351770,9.983790,0.406604,0.942611,0.412255,0.891026
max,9.949470,154.981630,21.077130,0.999989,0.999998,0.993901,1.000000


In [847]:
runtime= boto3.client('runtime.sagemaker')

In [848]:
i = random.randint(0,len(df_reordered_cos.index))
# boat_speed angle_of_attack wind_speed cos_target_angle sign_sin_target_angle
test_values_cos = [df_reordered_cos.iloc[i,:-2].to_numpy()]
#gt_sign_sin = max(df_reordered.iloc[i,-1], 0) # -1.0 was set to 0
gt_sin = df_reordered_cos.iloc[i,-1]
gt_cos = df_reordered_cos.iloc[i,-2]

print(test_values_cos)
test_values_sin = [df_reordered_sin.iloc[i,:-2].to_numpy()]
print(test_values_sin)

pprint(dict(zip(list(df_reordered.columns.values), df_reordered.iloc[i,:])))

[array([   3.60135   , -136.6674    ,    4.15544   ,   -0.81941255,
          0.57320421])]
[array([   3.60135   , -136.6674    ,    4.15544   ,    0.57320421,
         -0.81941255])]
{'angle_of_attack': -136.6674,
 'boat_speed': 3.60135,
 'cos_boat_angle': -0.6426529638821419,
 'cos_target_angle': -0.8194125517195305,
 'sin_boat_angle': 0.7661574042019684,
 'sin_target_angle': 0.5732042132473274,
 'wind_speed': 4.1554400000000005}


In [849]:
payload_cos = np2csv(test_values_cos)
payload_sin = np2csv(test_values_sin)
cos_endpoint ='VR-cos-regression-endpoint-202009251406'
sign_sin_endpoint = 'VR-sign-sin-classifier-endpoint-202009111431'
sin_endpoint = 'VR-sin-regression-endpoint-202009251406'


In [850]:
print(payload_cos)
response = runtime.invoke_endpoint(EndpointName=cos_endpoint,
                                   ContentType='text/csv',
                                   Body=payload_cos)
result = json.loads(response['Body'].read().decode())
print(result)
pred_cos = np.array([r['score'] for r in result['predictions']])[0]
pred_cos = min(1, pred_cos)
pred_cos = max(-1, pred_cos)

3.60135,-136.667,4.15544,-0.819413,0.573204
{'predictions': [{'score': -0.7308506965637207}]}


In [851]:
print(payload_sin)
response = runtime.invoke_endpoint(EndpointName=sin_endpoint,
                                   ContentType='text/csv',
                                   Body=payload_sin)
result = json.loads(response['Body'].read().decode())
print(result)
pred_sin = np.array([r['score'] for r in result['predictions']])[0]
pred_sin = min(1, pred_sin)
pred_sin = max(-1, pred_sin)

3.60135,-136.667,4.15544,0.573204,-0.819413
{'predictions': [{'score': 0.6392322778701782}]}


In [852]:

print(f'Ground Truth sinus: {round(gt_sin, 2)}')
print(f'Predicted sinus: {round(pred_sin,2)}')
print(f'Ground Truth cosinus: {round(gt_cos,2)}')
print(f'Predicted cosinus: {round(pred_cos,2)}')

if gt_sin > 0:
    print(f'Ground Truth angle {round(math.degrees(math.acos(gt_cos)),2)}')
else:
    print(f'Ground Truth angle {round(360 - math.degrees(math.acos(gt_cos)),2)}')
    
if pred_sin > 0:
    print(f'Predicted angle {round(math.degrees(math.acos(pred_cos)),2)}')
else:
    print(f'Predicted angle {round(360 - math.degrees(math.acos(pred_cos)),2)}')

Ground Truth sinus: 0.77
Predicted sinus: 0.64
Ground Truth cosinus: -0.64
Predicted cosinus: -0.73
Ground Truth angle 129.99
Predicted angle 136.96
